# Usando Middleware para añadir HITL en el Proceso Agéntico

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Lee un email desde la dirección proporcionada."""
    # obtener email del estado
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Envía un email a la dirección proporcionada con el asunto y cuerpo indicados."""
    # envío de email simulado
    return f"Email enviado"

from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware
from pprint import pprint

class EmailState(AgentState):
    email: str

agent = create_agent(
    model="gpt-4o-mini",
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="La ejecución de la herramienta requiere aprobación",
        ),
    ],
)


from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Por favor, lee mi email y envía una respuesta.")],
        "email": "Hola, Donald. Estaré en la ciudad mañana, ¿tendrás una habitación disponible en la Casa Blanca? Saludos, Julio."
    },
    config=config
)

print(response['__interrupt__'])


from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=config # Mismo thread ID para reanudar la conversación pausada
)

pprint(response)

response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # Una explicación de por qué se rechazó la solicitud
                    "message": "Lo siento, tenemos a Vlad Putin mañana para cenar y la Casa Blanca estará completa. Saludos, Donald."
                }
            ]
        }
    ), 
    config=config # Mismo thread ID para reanudar la conversación pausada
)   

pprint(response)

response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Acción editada con el nombre de la herramienta y los argumentos
                    "edited_action": {
                        # Nombre de la herramienta a llamar.
                        # Normalmente será el mismo que la acción original.
                        "name": "send_email",
                        # Argumentos a pasar a la herramienta.
                        "args": {"body": "¡Por supuesto! Serás nuestro invitado de honor. Saludos, Donald."},
                    }
                }
            ]
        }
    ), 
    config=config # Mismo thread ID para reanudar la conversación pausada
)   

pprint(response)

## ¡Muchas cosas están pasando ahí! Expliquemos el código anterior en términos sencillos
A continuación se muestra una explicación **sencilla, línea por línea** de lo que hace vuestro código, centrada en **Middleware + Humano en el Bucle (HITL)** para principiantes.

Idea clave primero: **el agente puede "pensar y elegir herramientas", pero el middleware HITL puede "pausar" justo antes de que se ejecuten ciertas herramientas, para que un humano pueda aprobar / rechazar / editar la acción.** 

---

#### 1) Importar los ayudantes de herramientas

```python
from langchain.tools import tool, ToolRuntime
```

* `tool` = un decorador que convierte una función Python normal en una **herramienta de LangChain** que el agente puede llamar.
* `ToolRuntime` = un objeto que LangChain pasa a las herramientas (cuando lo solicitas) que puede incluir cosas como **el estado del agente** y el contexto de ejecución.

---

#### 2) Herramienta #1: read_email (lee del estado del agente)

```python
@tool
def read_email(runtime: ToolRuntime) -> str:
    """Lee un email desde la dirección proporcionada."""
    # obtener email del estado
    return runtime.state["email"]
```

Línea por línea:

* `@tool` → "Hace esta función disponible como herramienta."
* `def read_email(runtime: ToolRuntime) -> str:`
  Esta herramienta recibe un objeto `runtime` (inyectado por LangChain).
* `return runtime.state["email"]`
  La herramienta busca dentro del **estado del agente** y devuelve el campo `"email"`.

Entonces: **esta herramienta no obtiene email real**—simplemente lee un valor almacenado en el estado.

(Usar campos de estado personalizados como este es exactamente para lo que sirve `state_schema`.)

---

#### 3) Herramienta #2: send_email (simula el envío)

```python
@tool
def send_email(body: str) -> str:
    """Envía un email a la dirección proporcionada con el asunto y cuerpo indicados."""
    # envío de email simulado
    return f"Email enviado"
```

Línea por línea:

* `@tool` → la hace invocable por el agente.
* `body: str` → el agente debe proporcionar un cuerpo de texto.
* devuelve `"Email enviado"` → es un stub (simulado). En la vida real, esto llamaría a Gmail/SMTP/etc.

---

#### 4) Importar agente + HITL + persistencia

```python
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware
from pprint import pprint
```

* `create_agent` = construye un agente listo para ejecutar que hace bucles: **pensar → llamada a herramienta → observar → pensar → …**
* `AgentState` = clase base para la memoria/estado a corto plazo del agente (como `messages`).
* `InMemorySaver` = un "checkpointer" que guarda el estado del agente para que pueda **pausar y reanudar** más tarde (necesario para interrupciones).
* `HumanInTheLoopMiddleware` = middleware que puede interrumpir llamadas a herramientas y requerir decisiones humanas.
* `pprint` = imprime objetos Python de forma bonita.

---

#### 5) Define tu estado de agente personalizado (añade `email`)

```python
class EmailState(AgentState):
    email: str
```

* Esto significa que el estado del agente incluirá todo de `AgentState` (como `messages`) **más** un campo `email`.
* El objetivo: las herramientas (como `read_email`) pueden leer `state["email"]`.

---

#### 6) Crear el agente + configurar la política HITL

```python
agent = create_agent(
    model="gpt-4o-mini",
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="La ejecución de la herramienta requiere aprobación",
        ),
    ],
)
```

Línea por línea (dentro de `create_agent`):

* `model="gpt-4o-mini"` → qué LLM usar.
* `tools=[read_email, send_email]` → las únicas acciones que el agente puede tomar.
* `state_schema=EmailState` → añade el campo `email` al estado del agente.
* `checkpointer=InMemorySaver()` → requerido porque HITL puede **pausar**, y necesitas estado guardado para **reanudar**.
* `middleware=[HumanInTheLoopMiddleware(...)]` → instala el "guardián" HITL.

Ahora la configuración HITL:

```python
interrupt_on={
    "read_email": False,
    "send_email": True,
},
```

* `"read_email": False` → deja que el agente lea sin preguntarte.
* `"send_email": True` → **pausa antes de enviar realmente** y pregunta a un humano qué hacer.

Esta es la idea central de HITL: **permitir herramientas seguras automáticamente, pero proteger herramientas arriesgadas** (como "enviar un email").

También:

```python
description_prefix="La ejecución de la herramienta requiere aprobación"
```

* Añade una etiqueta/mensaje amigable al aviso de interrupción mostrado al humano.

---

#### 7) Hacer una solicitud inicial al agente

```python
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}
```

* `HumanMessage` representa el mensaje del usuario.
* `thread_id="1"` es crucial: le dice a LangGraph/LangChain **"esta es la misma conversación en curso"**, para que puedas reanudar después de una interrupción usando el mismo thread. (Persistencia + identidad de thread es cómo funciona "pausar/reanudar".)

Ahora invocar:

```python
response = agent.invoke(
    {
        "messages": [HumanMessage(content="Por favor, lee mi email y envía una respuesta.")],
        "email": "Hola, Donald. Estaré en la ciudad mañana...."
    },
    config=config
)
```

Lo que significa esta entrada:

* `"messages": [...]` → el historial de conversación comienza con tu solicitud.
* `"email": "Hola, Donald...."` → estás colocando un email en el **estado** del agente (porque tu `EmailState` lo permite).

Entonces el agente puede:

1. llamar a `read_email` (permitido)
2. decidir sobre una respuesta
3. intentar llamar a `send_email` → **esto activa la pausa HITL** (porque interrupt_on dice True)

---

#### 8) Imprimir la carga de interrupción

```python
print(response['__interrupt__'])
```

* Cuando el middleware HITL pausa la ejecución, la respuesta devuelta contiene un campo `__interrupt__` que describe:

  * qué llamada a herramienta está a punto de ejecutarse,
  * qué argumentos quiere usar,
  * y qué decisiones puede proporcionar el humano.

Los documentos de LangChain describen explícitamente este comportamiento de "resultado interrumpido tiene `__interrupt__`".

---

#### 9) Reanudación #1: aprobar la llamada a herramienta

```python
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=config # Mismo thread ID para reanudar la conversación pausada
)

pprint(response)
```

Línea por línea:

* `Command(resume=...)` es cómo "continúas desde la pausa" proporcionando la decisión del humano.
* `decisions: [{"type": "approve"}]` significa:

  * "Ejecuta la herramienta exactamente como el agente propuso."
* Mismo `config` con el mismo `thread_id` → reanuda la ejecución pausada.

Entonces: **el email se "envía" (en tu herramienta simulada), y el agente continúa.** 

---

#### 10) Reanudación #2: rechazar, con mensaje de retroalimentación

```python
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # Una explicación de por qué se rechazó la solicitud
                    "message": "Lo siento, tenemos a Vlad Putin mañana para cenar y la Casa Blanca estará completa. Saludos, Donald."
                }
            ]
        }
    ), 
    config=config
)   

pprint(response)
```

Lo que esto hace:

* `"type": "reject"` → "NO ejecutes la herramienta."
* `"message": ...` → retroalimentación para el agente (como "esto es lo que debes decir en su lugar / por qué no lo estamos haciendo").

En términos HITL: rechazar = "saltar esta acción, y aquí está la orientación humana."

---

#### 11) Reanudación #3: editar la llamada a herramienta (cambiar argumentos)

```python
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Acción editada con el nombre de la herramienta y los argumentos
                    "edited_action": {
                        "name": "send_email",
                        "args": {"body": "¡Por supuesto! Serás nuestro invitado de honor. Saludos, Donald."},
                    }
                }
            ]
        }
    ), 
    config=config
)   

pprint(response)
```

Lo que esto hace:

* `"type": "edit"` → "Ejecuta la herramienta, pero NO con los argumentos originales del agente."
* `edited_action` permite al humano especificar:

  * el nombre de la herramienta (`send_email`)
  * los nuevos argumentos (`body=...`)

Entonces: el agente quería enviar algo, pero **tú anulas el contenido** antes de que "envíe."

---

#### Modelo mental: qué sucede de principio a fin

1. Llamas a `agent.invoke(...)`.
2. El agente piensa y llama a herramientas.
3. El middleware HITL verifica cada llamada a herramienta:

   * `read_email` → permitido automáticamente
   * `send_email` → **interrupción**
4. Inspeccionas `response["__interrupt__"]`.
5. Reanudas con una decisión `Command(resume=...)`:

   * `approve` → ejecutar tal cual
   * `reject` → saltar + retroalimentación
   * `edit` → ejecutar con argumentos cambiados

Eso es todo: **Middleware = capa de control; middleware HITL = "puerta de aprobación humana" para herramientas arriesgadas.**

## Bien, ese es un código interesante, pero si quisiéramos probar esta aplicación proporcionando retroalimentación humana real, ¿cómo debería ser el código?

Para hacer que esto funcione con **retroalimentación humana real**, necesitas separar la ejecución en fases y añadir una forma de capturar la entrada real del usuario. Así es como modificar el código:

## Primero, a continuación tienes la versión del código para ejecutar en tu Jupyter Notebook (esto NO funcionará si intentas ejecutarlo en Visual Studio Code)

In [ ]:
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain.messages import HumanMessage
from langgraph.types import Command
from pprint import pprint
from IPython.display import display, Markdown

# Definiciones de herramientas
@tool
def read_email(runtime: ToolRuntime) -> str:
    """Lee un email desde la dirección proporcionada."""
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Envía un email a la dirección proporcionada con el asunto y cuerpo indicados."""
    return f"✅ ¡Email enviado con éxito!\n\nCuerpo: {body}"

# Configuración del estado y agente
class EmailState(AgentState):
    email: str

agent = create_agent(
    model="gpt-4o-mini",
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="La ejecución de la herramienta requiere aprobación",
        ),
    ],
)

# Configuración
config = {"configurable": {"thread_id": "1"}}

# FASE 1: Invocación inicial
display(Markdown("## 🚀 FASE 1: Iniciando agente..."))

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Por favor, lee mi email y envía una respuesta.")],
        "email": "Hola, Donald. Estaré en la ciudad mañana, ¿tendrás una habitación disponible en la Casa Blanca? Saludos, Julio."
    },
    config=config
)

# Función para manejar interrupciones (podríamos necesitar manejar múltiples interrupciones)
def handle_interrupt(response, config, interrupt_number=1):
    """Maneja una única interrupción y devuelve la respuesta actualizada"""
    
    if '__interrupt__' not in response:
        return response, False  # Sin interrupción, hemos terminado
    
    display(Markdown(f"### ⚠️ INTERRUPCIÓN #{interrupt_number} - ¡Se requiere aprobación humana!"))
    
    # Extraer información de interrupción
    interrupt_obj = response['__interrupt__'][0]
    interrupt_value = interrupt_obj.value
    
    # Extraer información de la herramienta del valor de interrupción
    action_requests = interrupt_value.get('action_requests', [])
    if action_requests:
        action_request = action_requests[0]
        tool_name = action_request.get('name', 'desconocido')
        tool_args = action_request.get('args', {})
        description = action_request.get('description', '')
        
        display(Markdown(f"**🤖 El agente quiere llamar a:** `{tool_name}`"))
        display(Markdown(f"**📝 Con argumentos:**"))
        pprint(tool_args)
        
        if description:
            display(Markdown(f"**📄 Descripción:**"))
            print(description)
    else:
        print("⚠️ No se encontraron solicitudes de acción en la interrupción")
        return response, False
    
    # FASE 2: Obtener entrada humana
    display(Markdown("---"))
    display(Markdown(f"## 🤔 MOMENTO DE DECIDIR (Interrupción #{interrupt_number})"))
    display(Markdown("""
    **¿Qué quieres hacer?**
    - Escribe `1` para **Aprobar** - Deja que el agente ejecute según lo planificado
    - Escribe `2` para **Rechazar** - Detén la acción y proporciona retroalimentación
    - Escribe `3` para **Editar** - Modifica el contenido antes de ejecutar
    """))
    
    choice = input("Introduce tu elección (1/2/3): ").strip()
    
    # FASE 3: Reanudar según la decisión
    display(Markdown("---"))
    display(Markdown(f"## ⚡ REANUDANDO (Interrupción #{interrupt_number})..."))
    
    if choice == "1":
        # APROBAR
        display(Markdown("### ✅ Has **APROBADO** la acción"))
        new_response = agent.invoke(
            Command(resume={"decisions": [{"type": "approve"}]}),
            config=config
        )
        return new_response, True
        
    elif choice == "2":
        # RECHAZAR
        display(Markdown("### ❌ Has **RECHAZADO** la acción"))
        feedback = input("Introduce tu mensaje de retroalimentación: ").strip()
        new_response = agent.invoke(
            Command(
                resume={
                    "decisions": [
                        {
                            "type": "reject",
                            "message": feedback
                        }
                    ]
                }
            ),
            config=config
        )
        return new_response, True
        
    elif choice == "3":
        # EDITAR
        display(Markdown("### ✏️ Has elegido **EDITAR** la acción"))
        new_body = input("Introduce el nuevo cuerpo del email: ").strip()
        new_response = agent.invoke(
            Command(
                resume={
                    "decisions": [
                        {
                            "type": "edit",
                            "edited_action": {
                                "name": tool_name,
                                "args": {"body": new_body}
                            }
                        }
                    ]
                }
            ),
            config=config
        )
        return new_response, True
        
    else:
        display(Markdown("### ❌ Elección no válida"))
        return response, False

# Manejar todas las interrupciones en un bucle
interrupt_count = 0
max_interrupts = 5  # Límite de seguridad para prevenir bucles infinitos

while '__interrupt__' in response and interrupt_count < max_interrupts:
    interrupt_count += 1
    response, continued = handle_interrupt(response, config, interrupt_count)
    
    if not continued:
        break

# RESULTADO FINAL
display(Markdown("---"))
display(Markdown("## 🎉 RESULTADO FINAL"))

if '__interrupt__' in response:
    display(Markdown("⚠️ **Todavía tiene interrupciones pendientes** (alcanzado límite máximo o elección no válida)"))
    pprint(response['__interrupt__'])
else:
    display(Markdown("✅ **¡Proceso completado con éxito!**"))
    
    # Mostrar solo los mensajes finales (saltar los detalles de interrupción)
    if 'messages' in response:
        display(Markdown("### 📧 Mensajes Finales:"))
        for msg in response['messages'][-3:]:  # Mostrar últimos 3 mensajes
            print(f"\n{msg.type.upper()}: {msg.content if hasattr(msg, 'content') else msg}")

## Vale, expliquemos las partes más importantes del código anterior en términos sencillos

#### **🎯 La Visión General**

Este código crea un agente de IA que **se pausa antes de realizar acciones sensibles** (como enviar emails) y espera tu aprobación. Piensa en ello como tener un botón de "enviar" que debes pulsar antes de que un email salga.

---

#### **1. La Configuración del Middleware - El "Guardián"**

```python
middleware=[
    HumanInTheLoopMiddleware(
        interrupt_on={
            "read_email": False,  # ✅ Auto-aprobar (sin pausa)
            "send_email": True,   # ⚠️ ¡PAUSAR AQUÍ! (necesita aprobación)
        },
    ),
]
```

**Lo que esto significa:**
- `interrupt_on` es como un punto de control de seguridad
- `False` = "Adelante, no se necesita aprobación"
- `True` = "¡DETENTE! Espera la aprobación humana"
- En este caso: leer es seguro ✅, pero enviar necesita aprobación ⚠️

**Por qué importa:** ¡Este simple diccionario controla qué acciones son seguras frente a peligrosas!

---

#### **2. El Checkpointer - La Función "Guardar Partida"**

```python
checkpointer=InMemorySaver(),
```

**Lo que esto significa:**
- Como un punto de control en un videojuego que guarda tu progreso
- Cuando el agente se pausa, guarda TODO sobre la conversación
- Puedes volver más tarde y reanudar exactamente desde donde lo dejaste

**Por qué importa:** Sin esto, el agente no puede pausar y reanudar. ¡El punto de control es **obligatorio** para que HITL funcione!

---

#### **3. El Thread ID - Tu "Nombre de Conversación"**

```python
config = {"configurable": {"thread_id": "1"}}
```

**Lo que esto significa:**
- Cada conversación necesita un ID único (como "conversacion_1", "conversacion_2", etc.)
- Esto le dice al agente "guarda esta conversación bajo este nombre"
- Cuando reanudas, usas el MISMO thread_id para continuar esa conversación exacta

**Por qué importa:** ¡Si cambias el thread_id, comienzas una conversación completamente nueva!

---

#### **4. La Respuesta de Interrupción - Qué Sucede Cuando Se Pausa**

```python
if '__interrupt__' in response:
    # ¡El agente está pausado y esperando!
```

**Lo que esto significa:**
- Cuando el agente llega a una herramienta que necesita aprobación, `__interrupt__` aparece en la respuesta
- Es como una **bandera roja** 🚩 diciendo "¡Estoy esperando tu decisión!"
- Dentro de `__interrupt__` está toda la información sobre lo que el agente quiere hacer

**Por qué importa:** ¡Así es como SABES que el agente está pausado y esperando por ti!

---

#### **5. Extraer la Información de la Herramienta**

```python
interrupt_obj = response['__interrupt__'][0]
interrupt_value = interrupt_obj.value
action_requests = interrupt_value.get('action_requests', [])
action_request = action_requests[0]
tool_name = action_request.get('name')
tool_args = action_request.get('args')
```

**Lo que esto significa (paso a paso):**
1. Obtener el objeto de interrupción (la notificación de pausa)
2. Extraer su `value` (los detalles)
3. Buscar dentro de `action_requests` (lo que el agente quiere hacer)
4. Obtener el `name` de la herramienta (ej., "send_email")
5. Obtener los `args` de la herramienta (ej., el cuerpo del email)

**Por qué importa:** ¡Esto te dice EXACTAMENTE lo que el agente está intentando hacer, para que puedas tomar una decisión informada!

---

#### **6. Los Tres Tipos de Decisión - Tus Elecciones**

```python
# Opción 1: APROBAR
Command(resume={"decisions": [{"type": "approve"}]})

# Opción 2: RECHAZAR
Command(resume={"decisions": [{"type": "reject", "message": "..."}]})

# Opción 3: EDITAR
Command(resume={"decisions": [{"type": "edit", "edited_action": {...}}]})
```

**Lo que cada una significa:**

**APROBAR** ✅
- "Sí, haz exactamente lo que planeaste"
- La herramienta se ejecuta con los argumentos originales

**RECHAZAR** ❌
- "No, no hagas eso"
- Proporcionas retroalimentación explicando por qué
- El agente recibe tu mensaje y puede intentar algo más

**EDITAR** ✏️
- "Hazlo, pero cambia primero los detalles"
- Modificas los argumentos (ej., cambiar el cuerpo del email)
- La herramienta se ejecuta con TU versión modificada

**Por qué importa:** ¡Estas tres opciones te dan control completo sobre las acciones del agente!

---

#### **7. El Comando de Reanudación - Continuar Después de la Pausa**

```python
agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}),
    config=config  # ← ¡Mismo thread_id!
)
```

**Lo que esto significa:**
- `Command(resume=...)` le dice al agente "aquí está mi decisión, ¡continúa!"
- DEBES usar el mismo `config` (thread_id) para reanudar la conversación correcta
- El agente retoma exactamente desde donde lo dejó

**Por qué importa:** ¡Así es como "despausas" el agente después de tomar tu decisión!

---

#### **8. El Bucle de Interrupción - Manejar Múltiples Pausas**

```python
while '__interrupt__' in response and interrupt_count < max_interrupts:
    interrupt_count += 1
    response, continued = handle_interrupt(response, config, interrupt_count)
```

**Lo que esto significa:**
- A veces el agente se interrumpe MÁS DE UNA VEZ
- Este bucle sigue manejando interrupciones hasta que no haya más
- `max_interrupts` previene bucles infinitos (¡seguridad!)

**Por qué importa:** ¡Sin este bucle, solo manejarías la primera interrupción y te perderías el resto!

---

#### **🎓 Conclusiones Clave para Principiantes**

1. **Middleware = Guardia de Seguridad** 
   - Decide qué herramientas necesitan aprobación

2. **Checkpointer = Botón de Guardar**
   - Requerido para pausar y reanudar

3. **Thread ID = ID de Conversación**
   - Debe permanecer igual para reanudar correctamente

4. **`__interrupt__` = Bandera Roja** 🚩
   - Significa "¡Estoy pausado, necesito tu input!"

5. **Tres Decisiones = Tu Control**
   - Aprobar, Rechazar o Editar la acción

6. **Command(resume=...) = Botón de Reanudar**
   - Le dice al agente que continúe con tu decisión

7. **Bucle = Manejar Todas las Interrupciones**
   - Algunas acciones desencadenan múltiples pausas

---

## **💡 Analogía del Mundo Real**

Piensa en ello como un **asistente de la sala de correos**:

1. **Leer correo** (read_email) → Pueden hacer esto libremente ✅
2. **Enviar correo** (send_email) → Primero deben mostrártelo y obtener aprobación ⚠️
3. **Tú lo revisas** → Puedes aprobar, rechazar o editar la carta
4. **Ellos continúan** → Después de tu decisión, terminan la tarea

El middleware es la política de la empresa, el checkpointer es su bloc de notas (recordando dónde lo dejaron), y el thread_id es la tarea específica en la que están trabajando.

## Aquí están las partes específicas de Jupyter del código anterior

#### **1. Importaciones de Display de IPython**
```python
from IPython.display import display, Markdown
```
**Por qué es específico de Jupyter:**
- `display()` y `Markdown()` están **solo disponibles en Jupyter/IPython**
- No existen en scripts Python normales
- Renderizan salida enriquecida y formateada en celdas de notebook

**Lo que hacen:**
- `Markdown()` - Renderiza markdown (encabezados, negrita, listas) bellamente en Jupyter
- `display()` - Muestra el markdown renderizado en la salida

---

#### **2. Formato Markdown para Salida Enriquecida**
```python
display(Markdown("## 🚀 FASE 1: Iniciando agente..."))
display(Markdown("### ⚠️ INTERRUPCIÓN - Se requiere aprobación humana"))
display(Markdown("**🤖 El agente quiere llamar a:** `send_email`"))
```

**Por qué es específico de Jupyter:**
- En un script Python normal, usarías sentencias `print()`
- Markdown hace la salida **mucho más bonita** en Jupyter con:
  - Encabezados (`##`, `###`)
  - Texto en negrita (`**texto**`)
  - Formato de código (`` `código` ``)
  - Los emojis se renderizan bien

**Alternativa de Python normal:**
```python
print("=" * 60)
print("FASE 1: Iniciando agente...")
print("=" * 60)
```

---

#### **3. Cuadros de Entrada Interactivos**
```python
choice = input("Introduce tu elección (1/2/3): ").strip()
new_body = input("Introduce el nuevo cuerpo del email: ").strip()
feedback = input("Introduce tu mensaje de retroalimentación: ").strip()
```

**Por qué es comportamiento específico de Jupyter:**
- En Jupyter, `input()` crea un **cuadro de texto interactivo** debajo de la celda
- Escribes directamente en la interfaz del notebook
- En una terminal/consola, `input()` funciona diferente (indicador de línea de comandos)

**Cómo se ve en Jupyter:**
```
Introduce tu elección (1/2/3): [____aparece cuadro de texto aquí____]
```

---

#### **4. Ejecución en Una Sola Celda**
```python
# TODO EL CÓDIGO EN UNA CELDA ← ¡Importante para Jupyter!
```

**Por qué es específico de Jupyter:**
- El **flujo de trabajo completo debe ejecutarse en UNA celda** para que HITL funcione correctamente
- Dividir entre múltiples celdas perdería el estado
- El checkpointer y config necesitan permanecer en memoria durante todo el proceso

**Si lo divides:**
```python
# Celda 1
response = agent.invoke(...)  # Crea interrupción

# Celda 2 (¡NO FUNCIONA!)
response = agent.invoke(Command(resume=...))  # ¡Contexto perdido!
```

---

#### **5. Impresión Bonita con pprint**
```python
from pprint import pprint
pprint(response)  # Muestra diccionarios anidados con buen formato
```

**Por qué es amigable para Jupyter:**
- Aunque `pprint` también funciona en Python normal, Jupyter lo renderiza bellamente
- Jupyter formatea automáticamente diccionarios con secciones colapsables
- Mucho más fácil de leer estructuras anidadas complejas en Jupyter

---

#### **📋 Comparación Rápida: Jupyter vs Python Normal**

| Característica | Jupyter Notebook | Script Python Normal |
|---------|------------------|----------------------|
| Formato enriquecido | `display(Markdown("## Encabezado"))` | `print("ENCABEZADO")` |
| Cuadros de entrada | `input()` crea cuadro de texto | `input()` usa terminal |
| Salida | Formateado, bonito, colapsable | Solo texto plano |
| Ejecución de celda | Una celda para todo el flujo | Ejecutar como script completo |
| Retroalimentación visual | Emojis, colores, markdown | Texto plano |

---

#### **🔄 Para Convertir a Script Python Normal**

Si quisieras ejecutar esto en una **terminal** en lugar de Jupyter:

```python
# ELIMINAR estas importaciones específicas de Jupyter:
# from IPython.display import display, Markdown

# REEMPLAZAR todo display(Markdown(...)) con print():
# display(Markdown("## 🚀 FASE 1..."))
print("\n" + "="*60)
print("🚀 FASE 1: Iniciando agente...")
print("="*60 + "\n")

# Mantener input() - ¡funciona en ambos!
choice = input("Introduce tu elección (1/2/3): ").strip()

# pprint funciona en ambos, pero se ve más simple en terminal
from pprint import pprint
pprint(response)
```

---

#### **✅ Resumen**

**Los 3 elementos clave específicos de Jupyter:**

1. **`from IPython.display import display, Markdown`** - Para salida enriquecida
2. **`display(Markdown("..."))`** - Para formato bonito  
3. **Ejecución en una sola celda** - Mantén todo junto

**Lo que funciona en AMBOS:**
- `input()` - Crea cuadros de texto en Jupyter, indicadores en terminal
- `pprint()` - Funciona en todos lados, solo más bonito en Jupyter
- Toda la lógica de LangChain/agente - ¡100% lo mismo!

Las partes específicas de Jupyter son **solo para hacerlo lucir bien**. ¡La funcionalidad HITL real (middleware, interrupciones, reanudar) funciona igual en todas partes! 🎯

## Vale, ahora veamos la versión de este código para Visual Studio Code

Aquí está la **versión para terminal/script Python** del código HITL:

```python
"""
Agente de Email con Humano en el Bucle - Versión para Terminal
Ejecuta este script desde tu terminal: python hitl_email_agent.py
"""

from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain.messages import HumanMessage
from langgraph.types import Command
from pprint import pprint

# Definiciones de herramientas
@tool
def read_email(runtime: ToolRuntime) -> str:
    """Lee un email desde la dirección proporcionada."""
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Envía un email a la dirección proporcionada con el asunto indicado."""
    return f"✅ ¡Email enviado con éxito!\n\nCuerpo: {body}"

# Configuración del estado y agente
class EmailState(AgentState):
    email: str

agent = create_agent(
    model="gpt-4o-mini",
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="La ejecución de la herramienta requiere aprobación",
        ),
    ],
)

# Configuración
config = {"configurable": {"thread_id": "1"}}

# Función auxiliar para imprimir encabezados de sección
def print_header(text, symbol="="):
    """Imprime un encabezado formateado"""
    print("\n" + symbol * 70)
    print(f"  {text}")
    print(symbol * 70 + "\n")

def print_subheader(text):
    """Imprime un subencabezado formateado"""
    print("\n" + "-" * 70)
    print(f"  {text}")
    print("-" * 70)

# Función para manejar interrupciones
def handle_interrupt(response, config, interrupt_number=1):
    """Maneja una única interrupción y devuelve la respuesta actualizada"""
    
    if '__interrupt__' not in response:
        return response, False  # Sin interrupción, hemos terminado
    
    print_header(f"⚠️  INTERRUPCIÓN #{interrupt_number} - ¡Se requiere aprobación humana!", "!")
    
    # Extraer información de interrupción
    interrupt_obj = response['__interrupt__'][0]
    interrupt_value = interrupt_obj.value
    
    # Extraer información de la herramienta del valor de interrupción
    action_requests = interrupt_value.get('action_requests', [])
    if action_requests:
        action_request = action_requests[0]
        tool_name = action_request.get('name', 'desconocido')
        tool_args = action_request.get('args', {})
        description = action_request.get('description', '')
        
        print(f"🤖 El agente quiere llamar a: {tool_name}")
        print(f"\n📝 Con argumentos:")
        pprint(tool_args)
        
        if description:
            print(f"\n📄 Descripción:")
            print(description)
    else:
        print("⚠️  No se encontraron solicitudes de acción en la interrupción")
        return response, False
    
    # Obtener entrada humana
    print_subheader(f"🤔 MOMENTO DE DECIDIR (Interrupción #{interrupt_number})")
    print("\n¿Qué quieres hacer?")
    print("  1 - Aprobar: Deja que el agente ejecute según lo planificado")
    print("  2 - Rechazar: Detén la acción y proporciona retroalimentación")
    print("  3 - Editar: Modifica el contenido antes de ejecutar")
    
    choice = input("\n👉 Introduce tu elección (1/2/3): ").strip()
    
    # Reanudar según la decisión
    print_subheader(f"⚡ REANUDANDO (Interrupción #{interrupt_number})...")
    
    if choice == "1":
        # APROBAR
        print("\n✅ Has APROBADO la acción\n")
        new_response = agent.invoke(
            Command(resume={"decisions": [{"type": "approve"}]}),
            config=config
        )
        return new_response, True
        
    elif choice == "2":
        # RECHAZAR
        print("\n❌ Has RECHAZADO la acción")
        feedback = input("👉 Introduce tu mensaje de retroalimentación: ").strip()
        print()
        new_response = agent.invoke(
            Command(
                resume={
                    "decisions": [
                        {
                            "type": "reject",
                            "message": feedback
                        }
                    ]
                }
            ),
            config=config
        )
        return new_response, True
        
    elif choice == "3":
        # EDITAR
        print("\n✏️  Has elegido EDITAR la acción")
        new_body = input("👉 Introduce el nuevo cuerpo del email: ").strip()
        print()
        new_response = agent.invoke(
            Command(
                resume={
                    "decisions": [
                        {
                            "type": "edit",
                            "edited_action": {
                                "name": tool_name,
                                "args": {"body": new_body}
                            }
                        }
                    ]
                }
            ),
            config=config
        )
        return new_response, True
        
    else:
        print("\n❌ Elección no válida\n")
        return response, False


def main():
    """Función principal para ejecutar el agente HITL"""
    
    # FASE 1: Invocación inicial
    print_header("🚀 FASE 1: Iniciando Agente con Humano en el Bucle")
    
    print("Enviando mensaje inicial al agente...")
    print("Contenido del email: 'Hola, Donald. Estaré en la ciudad mañana, ¿tendrás ")
    print("una habitación disponible en la Casa Blanca? Saludos, Julio.'\n")
    
    response = agent.invoke(
        {
            "messages": [HumanMessage(content="Por favor, lee mi email y envía una respuesta.")],
            "email": "Hola, Donald. Estaré en la ciudad mañana, ¿tendrás una habitación disponible en la Casa Blanca? Saludos, Julio."
        },
        config=config
    )
    
    # Manejar todas las interrupciones en un bucle
    interrupt_count = 0
    max_interrupts = 5  # Límite de seguridad para prevenir bucles infinitos
    
    while '__interrupt__' in response and interrupt_count < max_interrupts:
        interrupt_count += 1
        response, continued = handle_interrupt(response, config, interrupt_count)
        
        if not continued:
            break
    
    # RESULTADO FINAL
    print_header("🎉 RESULTADO FINAL")
    
    if '__interrupt__' in response:
        print("⚠️  Todavía tiene interrupciones pendientes (alcanzado límite máximo o elección no válida)\n")
        print("Detalles de interrupción:")
        pprint(response['__interrupt__'])
    else:
        print("✅ ¡Proceso completado con éxito!\n")
        
        # Mostrar solo los mensajes finales
        if 'messages' in response:
            print("📧 Mensajes Finales:")
            print("-" * 70)
            for msg in response['messages'][-3:]:  # Mostrar últimos 3 mensajes
                msg_type = msg.type.upper() if hasattr(msg, 'type') else 'MENSAJE'
                msg_content = msg.content if hasattr(msg, 'content') else str(msg)
                print(f"\n[{msg_type}]")
                print(msg_content)
                print()


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\n⚠️  Interrumpido por el usuario. Saliendo...")
    except Exception as e:
        print(f"\n\n❌ Ocurrió un error: {e}")
        import traceback
        traceback.print_exc()
```

---

## **Diferencias Clave de la Versión Jupyter:**

#### **1. Importaciones Específicas de Jupyter Eliminadas**
```python
# ELIMINADO:
# from IPython.display import display, Markdown
```

#### **2. Markdown Reemplazado con Sentencias Print**
```python
# ANTES (Jupyter):
display(Markdown("## 🚀 FASE 1: Iniciando agente..."))

# DESPUÉS (Terminal):
print_header("🚀 FASE 1: Iniciando Agente con Humano en el Bucle")
```

#### **3. Funciones Auxiliares Añadidas para Formato**
```python
def print_header(text, symbol="="):
    """Imprime un encabezado formateado"""
    print("\n" + symbol * 70)
    print(f"  {text}")
    print(symbol * 70 + "\n")
```
- Crea separadores visuales agradables en la terminal
- Hace la salida más legible

#### **4. Todo Envuelto en main()**
```python
def main():
    """Función principal para ejecutar el agente HITL"""
    # ... toda la lógica aquí ...

if __name__ == "__main__":
    main()
```
- Práctica estándar de Python para scripts
- Lo hace reutilizable e importable
- Estructura más limpia

#### **5. Manejo de Errores Añadido**
```python
try:
    main()
except KeyboardInterrupt:
    print("\n\n⚠️  Interrumpido por el usuario. Saliendo...")
except Exception as e:
    print(f"\n\n❌ Ocurrió un error: {e}")
```
- Maneja Ctrl+C con gracia
- Muestra errores correctamente en terminal

---

## **Ejemplo de Salida en Terminal:**

```
======================================================================
  🚀 FASE 1: Iniciando Agente con Humano en el Bucle
======================================================================

Enviando mensaje inicial al agente...
Contenido del email: 'Hola, Donald. Estaré en la ciudad mañana, ¿tendrás
una habitación disponible en la Casa Blanca? Saludos, Julio.'

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
  ⚠️  INTERRUPCIÓN #1 - ¡Se requiere aprobación humana!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

🤖 El agente quiere llamar a: send_email

📝 Con argumentos:
{'body': 'Hola Julio,\n\nSí, tendremos una habitación disponible...'}

----------------------------------------------------------------------
  🤔 MOMENTO DE DECIDIR (Interrupción #1)
----------------------------------------------------------------------

¿Qué quieres hacer?
  1 - Aprobar: Deja que el agente ejecute según lo planificado
  2 - Rechazar: Detén la acción y proporciona retroalimentación
  3 - Editar: Modifica el contenido antes de ejecutar

👉 Introduce tu elección (1/2/3): 1

----------------------------------------------------------------------
  ⚡ REANUDANDO (Interrupción #1)...
----------------------------------------------------------------------

✅ Has APROBADO la acción

======================================================================
  🎉 RESULTADO FINAL
======================================================================

✅ ¡Proceso completado con éxito!

📧 Mensajes Finales:
----------------------------------------------------------------------

[AI]
...

[TOOL]
✅ ¡Email enviado con éxito!

Cuerpo: Hola Julio,...
```

---

## Beneficios de Esta Versión para Terminal

✅ **Se ejecuta en cualquier lugar** - No se necesita Jupyter
✅ **Estructura más limpia** - Usa funciones y main()
✅ **Mejor manejo de errores** - Captura Ctrl+C y excepciones
✅ **Formato profesional** - Encabezados y separadores agradables
✅ **Fácil de desplegar** - Se puede ejecutar en servidores, CI/CD, etc.
✅ **Reutilizable** - Se pueden importar funciones en otros scripts

¡Esta versión hace **exactamente lo mismo** que la versión Jupyter, solo que optimizada para ejecución en terminal! 🚀

## Cómo ejecutar este código desde Visual Studio Code
* Abre la Terminal.
* Asegúrate de estar en la carpeta del proyecto.
* Asegúrate de tener el entorno de poetry activado.
* Introduce y ejecuta el siguiente comando:
    * `python 013-mid-to-add-HITL.py`